In [ ]:
%pip install requests
%pip install pandas
%pip install beautifulsoup4
%pip install boto3
%pip install logging

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import logging
from urllib.parse import urlparse

def lambda_handler(event, context):
    
    return {
        'statusCode': 200,
        'body': 'donga-news-crawler from Lambda is do!'
    }

logging.basicConfig(level='DEBUG')

header = {'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                         '(KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'),}

categories = {
    '정치': 'https://www.donga.com/news/Politics',
    '경제': 'https://www.donga.com/news/Economy',
    '국제': 'https://www.donga.com/news/Inter',
    '사회_사건': 'https://www.donga.com/news/Society/Event', # 앞 분기 기준
    '경제_과학': 'https://www.donga.com/news/It/List', # 앞 분기 기준
    '사회': 'https://www.donga.com/news/Society',
    '스포츠': 'https://www.donga.com/news/Sports',
}

result_categories = {
    '정치': 'politics',
    '경제': 'economy',
    '국제': 'world',
    '사회_사건': 'accident',
    '경제_과학': 'science',
    '사회': 'society',
    '스포츠': 'sports',
}

def scrap_article(article_url, category):
    response = requests.get(article_url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        page_category = soup.find('ol', class_='breadcrumb').find('li', class_='breadcrumb_item').find('a').get_text(strip=True)
        logging.info(f"page_category: {page_category}")
        
        logging.info(f"category: {category}")

        category_parts = category.split('_')
        
        # 카테고리가 아닌 글은 패스
        if page_category not in category_parts:
            logging.info(f"{article_url} 는 해당 카테고리가 아니어서 생략합니다. ")
            return None
    except AttributeError:
        logging.error(f"{article_url} 는 찾을 수 없습니다.")
        return None

    # 헤드라인 스크랩
    try:
        title = soup.find('section', {'class': 'head_group'}).find('h1').get_text(strip=True)
    except AttributeError:
        title = 'N/A'
        
    try:
        date_button = soup.find('button', {'aria-controls': 'dateInfo'})
        time_span = date_button.find('span', {'aria-hidden': 'true'})
        time = time_span.get_text(strip=True)  # "2024-09-30 21:44" 형태
        time = time.replace(" ", "T") + ":00+09:00"  # ISO 8601 형식으로 변환 (예: 2024-09-30T21:44:00+09:00)
    except AttributeError:
        time = 'N/A'

    # 본문 스크랩
    try:
        # 광고, 이미지 등 제외하고 텍스트만 추출
        content_section = soup.find('section', class_='news_view')
        for tag in content_section(['figure', 'script', 'div', 'ul']):  # 불필요한 태그 제거
            tag.decompose()
        content = content_section.get_text(separator="\n", strip=True)
    except AttributeError:
        content = 'N/A'

    result_category = result_categories.get(category, 'unknown')

    return {'title': title, 'time': time, 'content': content, 'source_url': article_url, 'category': result_category}

def post_article_to_server(article_data):
    
    fastapi_url = 'https://your-fastapi-url.com/process_data'
    headers = {'Content-Type': 'application/json'}
    
    try:
        response = requests.post(fastapi_url, json=article_data, headers=headers)
        response.raise_for_status()
        logging.info(f'\'statusCode\': 200 \n\'body\': \'요청 성공\'')
    except requests.exceptions.RequestException as e:
        logging.error(f'\'statusCode\': 500 \n\'Internal Server Error\': {e}')

def check_for_new_articles(base_url, category):
    response = requests.get(base_url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    article_count = 0
    cards = soup.find_all('article', class_='news_card')
    cards.reverse()
    for card in cards:
        if article_count >= 3:
            break

        link_tag = card.find('a', href=True)
        if link_tag and 'href' in link_tag.attrs:
            article_link = link_tag['href']
        else:
            logging.warning("article_link is None or missing 'href'.")
            continue
    
        logging.debug(f"Scraping article: {article_link}")

        result = scrap_article(article_link, category)

        if result is None:
            continue
        
        # post_article_to_server(result) # 서버 완성되면 주석을 풀어라...
        print(result)
        article_count += 1

def scrap_all_categories():
    
    for category, base_url in categories.items():
        logging.info(f"category (base_url): {category} ({base_url})")
        check_for_new_articles(base_url, category)
        time.sleep(2)

scrap_all_categories()